# Source Extration for 2-Photon Calcium Imaging Data

## Setting Parameters for Source Extraction and Motion Correction for 2p 

## CNMF for Source Extraction

## Refitting CNMF Algorithm

## Exploring the Estimates Class